In [2]:
# Importa as bibliotecas necessárias
import pandas as pd                                # Para trabalhar com tabelas
import requests                                    # Para fazer chamadas à API
import time                                        # Para controlar o tempo entre chamadas
from urllib.parse import quote_plus               # Para codificar nomes em URLs
from requests.exceptions import RequestException  # Para capturar erros da API
from unidecode import unidecode                   # Para remover acentos das cidades

# Leitura do arquivo CSV com os dados dos países
df = pd.read_csv("df_fato_pais.csv")  # Certifique-se de que o arquivo está no mesmo diretório do script

# Filtra os dados válidos: capital e código do país (sigla de 2 letras como BR, US, etc.)
cidades_paises = [
    (row['capital'], row['nome_resumido'])
    for _, row in df.iterrows()
    if (
        pd.notna(row['capital']) and                      # Capital não pode ser ausente
        pd.notna(row['nome_resumido']) and                # Código do país também não
        row['capital'].strip().lower() != 'não informado' and
        row['capital'].strip() != ''                      # Capital não pode estar vazia
    )
]

#  Remove duplicatas, caso algum país tenha aparecido mais de uma vez
cidades_paises = list(set(cidades_paises))

# Lista onde vamos guardar os dados climáticos coletados
dados_clima = []

# Lista de capitais que falharam ao buscar o clima (útil para revisar depois)
capitais_falharam = []

# Sua chave da API da OpenWeatherMap (você precisa se cadastrar no site deles para conseguir)
api_key = "e0b696d684ea5b8a15f7f141ca979bc3"  # <<<<<<<<<<<<<<< Substitua isso pela sua chave real

# Loop para buscar o clima de cada cidade
for cidade_original, codigo_pais in cidades_paises:

    #Remove acentos da cidade (ex: "Brasília" → "Brasilia")
    cidade_sem_acento = unidecode(cidade_original)

    # Codifica para URL (ex: "San Jose" → "San+Jose")
    cidade_codificada = quote_plus(cidade_sem_acento)

    try:
        # Monta os parâmetros para chamada da API
        url = "https://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": f"{cidade_codificada},{codigo_pais}",  # Ex: "Brasilia,BR"
            "appid": api_key,
            "units": "metric",                          # Para temperatura em graus Celsius
            "lang": "pt_br"                             # Retorno em português
        }

        # Chama a API com timeout de 10 segundos
        resposta = requests.get(url, params=params, timeout=10)
        resposta.raise_for_status()  # Se a resposta for erro (ex: 404), levanta exceção

        # Converte a resposta JSON da API para um dicionário Python
        clima_info = resposta.json()

        # Salva os dados relevantes da resposta na lista
        dados_clima.append({
            "cidade": cidade_original,
            "codigo_pais": clima_info['sys']['country'],
            "temperatura": clima_info['main']['temp'],
            "sensacao_termica": clima_info['main']['feels_like'],
            "temperatura_min": clima_info['main']['temp_min'],
            "temperatura_max": clima_info['main']['temp_max'],
            "umidade": clima_info['main']['humidity'],
            "descricao_clima": clima_info['weather'][0]['description'],
            "horario_medicao": clima_info['dt']  # Timestamp do horário da medição
        })

    except RequestException as e:
        # Caso ocorra erro da API (ex: cidade não encontrada)
        print(f"[ERRO API] {cidade_original}, {codigo_pais}: {e}")
        capitais_falharam.append((cidade_original, codigo_pais))

    except Exception as e:
        # Qualquer outro erro inesperado
        print(f"[ERRO INESPERADO] {cidade_original}, {codigo_pais}: {e}")
        capitais_falharam.append((cidade_original, codigo_pais))

    # ⏱Espera 1 segundo para não ultrapassar limite da API
    time.sleep(1)

# 8. Converte a lista de dados em uma tabela
df_clima = pd.DataFrame(dados_clima)

# 9. Mostra os dados climáticos coletados
print("\nDados coletados:")
print(df_clima)

# 10. Exibe as capitais que não foram encontradas
print("\nCapitais com erro:")
for cidade, pais in capitais_falharam:
    print(f" - {cidade} ({pais})")

# 11. (Opcional) Salva os dados em arquivos CSV
df_clima.to_csv("df_clima_capitais.csv", index=False)
pd.DataFrame(capitais_falharam, columns=["cidade", "codigo_pais"]).to_csv("df_falhas_clima.csv", index=False)


[ERRO API] Saint Helier, JE: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Saint%2BHelier%2CJE&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] San Juan, PR: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=San%2BJuan%2CPR&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] Pago Pago, AS: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Pago%2BPago%2CAS&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] Diego Garcia, IO: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Diego%2BGarcia%2CIO&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] The Valley, AI: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=The%2BValley%2CAI&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] São Tomé, ST

In [3]:

display(df_clima)


,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Bishkek,KG,19.11,18.15,19.11,19.11,41,nublado,1758910311
1,Pretoria,ZA,22.49,21.89,22.49,22.49,42,algumas nuvens,1758910115
2,Bucharest,RO,14.10,12.77,13.25,15.05,46,céu limpo,1758910202
3,Lomé,TG,25.97,25.97,25.97,25.97,83,nublado,1758910357
4,Tashkent,UZ,20.97,20.04,20.97,20.97,35,fumaça,1758910359
...,...,...,...,...,...,...,...,...,...
191,Tehran,IR,24.84,23.88,24.84,24.99,19,algumas nuvens,1758910484
192,Lisbon,PT,22.09,21.45,20.84,22.53,42,céu limpo,1758910700
193,Fort-de-France,MQ,33.97,39.02,33.97,33.97,52,nublado,1758910535
194,Papeetē,PF,27.96,29.96,27.73,27.96,65,chuva leve,1758910511


In [4]:
from pandasql import sqldf

# Define uma função para executar consultas SQL
pysqldf = lambda q: sqldf(q, globals())

In [5]:
consultaSQL = """

SELECT * FROM df_clima 
--where codigo_pais = "AR"

"""

resultado = pysqldf(consultaSQL)
display(resultado)

,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Bishkek,KG,19.11,18.15,19.11,19.11,41,nublado,1758910311
1,Pretoria,ZA,22.49,21.89,22.49,22.49,42,algumas nuvens,1758910115
2,Bucharest,RO,14.10,12.77,13.25,15.05,46,céu limpo,1758910202
3,Lomé,TG,25.97,25.97,25.97,25.97,83,nublado,1758910357
4,Tashkent,UZ,20.97,20.04,20.97,20.97,35,fumaça,1758910359
...,...,...,...,...,...,...,...,...,...
191,Tehran,IR,24.84,23.88,24.84,24.99,19,algumas nuvens,1758910484
192,Lisbon,PT,22.09,21.45,20.84,22.53,42,céu limpo,1758910700
193,Fort-de-France,MQ,33.97,39.02,33.97,33.97,52,nublado,1758910535
194,Papeetē,PF,27.96,29.96,27.73,27.96,65,chuva leve,1758910511


In [6]:
resultado['horario_medicao'] = pd.to_datetime(resultado['horario_medicao'], unit='s').dt.strftime('%d/%m/%Y %H:%M:%S')

In [7]:
display(resultado)

,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Bishkek,KG,19.11,18.15,19.11,19.11,41,nublado,26/09/2025 18:11:51
1,Pretoria,ZA,22.49,21.89,22.49,22.49,42,algumas nuvens,26/09/2025 18:08:35
2,Bucharest,RO,14.10,12.77,13.25,15.05,46,céu limpo,26/09/2025 18:10:02
3,Lomé,TG,25.97,25.97,25.97,25.97,83,nublado,26/09/2025 18:12:37
4,Tashkent,UZ,20.97,20.04,20.97,20.97,35,fumaça,26/09/2025 18:12:39
...,...,...,...,...,...,...,...,...,...
191,Tehran,IR,24.84,23.88,24.84,24.99,19,algumas nuvens,26/09/2025 18:14:44
192,Lisbon,PT,22.09,21.45,20.84,22.53,42,céu limpo,26/09/2025 18:18:20
193,Fort-de-France,MQ,33.97,39.02,33.97,33.97,52,nublado,26/09/2025 18:15:35
194,Papeetē,PF,27.96,29.96,27.73,27.96,65,chuva leve,26/09/2025 18:15:11


In [8]:
df_export = pd.DataFrame(resultado)  # Dados dim pais
df_export.to_csv("df_dim_tempo_capital.csv", index=False)

In [9]:
# ==========================================================
# Importando bibliotecas necessárias
# ==========================================================
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv, find_dotenv

# Carrega as variáveis de ambiente a partir do arquivo .env
load_dotenv(find_dotenv())

# ==========================================================
# Definindo as credenciais do banco (Render)
# ==========================================================
host = os.getenv("HOST")
port = int(os.getenv("PORT"))
database = os.getenv("DATABASE")
user = os.getenv("USER")
password = os.getenv("PASSWORD")

# ==========================================================
# Criando a conexão com o PostgreSQL usando SQLAlchemy
# ==========================================================
engine = create_engine(
    f"postgresql://{user}:{password}@{host}:{port}/{database}"
)


# ==========================================================
# Salvando no banco (tabela: clima)
# if_exists:
#   - 'replace' -> apaga e recria a tabela
#   - 'append' -> adiciona os dados
# ==========================================================
df_export.to_sql("tbl_clima_capital_pais", engine, if_exists="replace", index=False)

print("Dados salvos no banco Render com sucesso!")

Dados salvos no banco Render com sucesso!


In [10]:
import psycopg2

# Parâmetros de conexão
dbname   = 'projeto_paises'
user     = 'postgres'
password = '252500'
host     = 'localhost'
port     = '5432' 

# Criar uma conexão
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)


cur = conn.cursor()# Criar um cursor  deixa manipular os dados
#cur.execute("DELETE FROM TBL_CLIMA_CAPITAL_PAIS")

for indice, colunas_df in resultado.iterrows():
        cur.execute('''                                
                insert into TBL_CLIMA_CAPITAL_PAIS (   
                 cidade,
                 codigo_pais,
                 temperatura,
                 sensacao_termica,
                 temperatura_min,
                 temperatura_max,
                 umidade,
                 descricao_clima,
                 horario_medicao)
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
                        ''',
        (
        colunas_df["cidade"],
        colunas_df["codigo_pais"],
        colunas_df["temperatura"],
        colunas_df["sensacao_termica"],
        colunas_df["temperatura_min"],
        colunas_df["temperatura_max"],
        colunas_df["umidade"],
        colunas_df["descricao_clima"],
        colunas_df["horario_medicao"]
        )
        )

#comandos sql transct-sql
conn.commit() # validar alterações que agente fez  e subir para o banco de dados 

# Fechar o cursor e a conexão
cur.close()
conn.close()